In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

path = "../../../dane/8CPU_20RAM/3600s/all_merged.csv"

In [2]:
# Załaduj dane
df = pd.read_csv(path)

In [3]:
for col in ['replicaId', 'endpointUrl_methods']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [4]:
# Dodajemy skalowanie dla wybranych kolumn
features_to_scale = ['queueSizeForward_methods', 'queueSizeBack_methods',
                     'cpuUsage_stock', 'memoryUsage_stock',
                     'applicationTime_trading', 'databaseTime_trading',
                     'numberOfSellOffers_trading', 'numberOfBuyOffers_trading']
scaler = StandardScaler()
for col in features_to_scale:
    df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

In [5]:
# Wybór cech
features = [
            # 'timestamp',
            # 'apiTime_methods',
            # 'applicationTime_methods',
            # 'databaseTime_methods',
            # 'endpointUrl_methods',
            # 'queueSizeForward_methods',
            # 'queueSizeBack_methods',
            'cpuUsage_stock',
            'memoryUsage_stock',
            # 'applicationTime_trading',
            # 'databaseTime_trading',
            # 'numberOfSellOffers_trading',
            # 'numberOfBuyOffers_trading',
            # 'cpuUsage_traffic',
            # 'memoryUsage_traffic',
            # 'replicaId'
            ]
df_features = df[features]

# Kodowanie pudełek
le = LabelEncoder()
df['test'] = le.fit_transform(df['test'])

# Kopiowanie danych
df_encoded = df_features.copy()
df_encoded['test'] = df['test']

In [6]:
# Stwórz puste listy do przechowywania danych treningowych i testowych
X_train = []
X_test = []
y_train = []
y_test = []

In [7]:
def create_windows(X, y, window_size, step_size):
    X_windows = []
    y_windows = []

    # Przesuń okno po danych
    for i in range(0, len(X) - window_size, step_size):
        # Utwórz okno danych
        X_window = X.iloc[i:i + window_size]
        # Utwórz etykietę dla okna (etykieta ostatniej obserwacji w oknie)
        y_window = y.iloc[i + window_size]

        X_windows.append(X_window.values)
        y_windows.append(y_window)

    return np.array(X_windows), np.array(y_windows)

In [8]:
window_size = 700
step_size = 100
# Dla każdego unikalnego pudełka
for box in df_encoded['test'].unique():
    # Wybierz tylko rekordy dla tego pudełka
    box_data = df_encoded[df_encoded['test'] == box]
    
    # Oblicz punkt podziału (80% danych)
    split_point = int(len(box_data) * 0.8)
    
    # Dodaj pierwsze 80% rekordów do danych treningowych
    X_train_box = box_data.drop('test', axis=1).iloc[:split_point]
    y_train_box = box_data['test'].iloc[:split_point]
    
    # Dodaj ostatnie 20% rekordów do danych testowych
    X_test_box = box_data.drop('test', axis=1).iloc[split_point:]
    y_test_box = box_data['test'].iloc[split_point:]
    
    # Stwórz okienka dla danych treningowych
    X_train_windows, y_train_windows = create_windows(X_train_box, y_train_box, window_size, step_size)
    
    # Stwórz okienka dla danych testowych
    X_test_windows, y_test_windows = create_windows(X_test_box, y_test_box, window_size, step_size)
    
    # Dodaj okienka do list
    X_train.append(X_train_windows)
    y_train.append(y_train_windows)
    X_test.append(X_test_windows)
    y_test.append(y_test_windows)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from keras.utils import to_categorical

In [10]:
# Połączenie danych treningowych dla wszystkich pudełek
X_train_combined = np.concatenate(X_train, axis=0)
y_train_combined = np.concatenate(y_train, axis=0)

In [11]:
# Zakodowanie etykiet w formacie one-hot encoding
y_train_combined_encoded = to_categorical(y_train_combined, num_classes=len(df['test'].unique()))

In [12]:
# Trenowanie modelu na połączonym zestawie danych
model = Sequential()    
model.add(GRU(50, return_sequences=False, input_shape=(window_size, X_train_combined.shape[2])))
model.add(Dense(len(df['test'].unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_combined, y_train_combined_encoded, epochs=25, batch_size=32)

Epoch 1/25
2023/2023 [==============================] - 304s 148ms/step - loss: 2.3107 - accuracy: 0.2588
Epoch 2/25
2023/2023 [==============================] - 346s 171ms/step - loss: 2.2810 - accuracy: 0.2603
Epoch 3/25
2023/2023 [==============================] - 322s 159ms/step - loss: 2.2424 - accuracy: 0.2602
Epoch 4/25
2023/2023 [==============================] - 291s 144ms/step - loss: 2.2145 - accuracy: 0.2763
Epoch 5/25
2023/2023 [==============================] - 290s 143ms/step - loss: 2.1495 - accuracy: 0.3030
Epoch 6/25
2023/2023 [==============================] - 288s 143ms/step - loss: 2.1067 - accuracy: 0.3225
Epoch 7/25
2023/2023 [==============================] - 289s 143ms/step - loss: 2.0513 - accuracy: 0.3390
Epoch 8/25
2023/2023 [==============================] - 289s 143ms/step - loss: 1.9662 - accuracy: 0.3521
Epoch 9/25
2023/2023 [==============================] - 290s 143ms/step - loss: 1.9774 - accuracy: 0.3527
Epoch 10/25
2023/2023 [=======================

In [13]:
for i in range(len(X_test)):
    y_test_encoded = to_categorical(y_test[i], num_classes=len(df['test'].unique()))
    loss, accuracy = model.evaluate(X_test[i], y_test_encoded)
    print(f"Test {i+1}: Loss = {loss}, Accuracy = {accuracy}")

34/34 [==============================] - 2s 37ms/step - loss: 1.1171 - accuracy: 0.6504
Test 1: Loss = 1.117135763168335, Accuracy = 0.6504217386245728
34/34 [==============================] - 1s 37ms/step - loss: 2.3715 - accuracy: 0.2744
Test 2: Loss = 2.3714725971221924, Accuracy = 0.2744360864162445
31/31 [==============================] - 1s 37ms/step - loss: 2.6895 - accuracy: 0.0875
Test 3: Loss = 2.689528465270996, Accuracy = 0.08748728036880493
33/33 [==============================] - 1s 37ms/step - loss: 2.5898 - accuracy: 0.1312
Test 4: Loss = 2.58980131149292, Accuracy = 0.13117870688438416
26/26 [==============================] - 1s 38ms/step - loss: 2.3924 - accuracy: 0.0000e+00
Test 5: Loss = 2.39241099357605, Accuracy = 0.0
29/29 [==============================] - 1s 37ms/step - loss: 2.0911 - accuracy: 0.3020
Test 6: Loss = 2.091078758239746, Accuracy = 0.3020496368408203
16/16 [==============================] - 1s 36ms/step - loss: 1.8866 - accuracy: 0.2293
Test 7: Lo